In [1]:
import pandas as pd

#Read the Input CSV File
df = pd.read_csv("D:/downloads/nikhil_kuttu/internshala/peoplebox/input.csv")

In [2]:
df

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01


In [3]:
#Sort Data by Employee Code and Date of Joining
df['Date of Joining'] = pd.to_datetime(df['Date of Joining'])
df['Date of Exit'] = pd.to_datetime(df['Date of Exit'])
df.sort_values(by=['Employee Code', 'Date of Joining'], inplace=True)

In [4]:
#Derive Effective and End Dates
df['End Date'] = df.groupby('Employee Code')['Date of Joining'].shift(-1) - pd.Timedelta(days=1)
df['End Date'].fillna(pd.Timestamp('2100-01-01'), inplace=True)

In [5]:
#Transform Columnar Data
output_df = pd.DataFrame(columns=['Employee Code', 'Manager Employee', 'Last Compensation', 'Compensation',
                                   'Last Pay Raise Date', 'Variable Pay', 'Tenure in Org',
                                   'Performance Rating', 'Engagement Score', 'Effective Date', 'End Date'])

In [7]:
for idx, row in df.iterrows():
    # Assuming compensation, pay raise date, variable pay, performance rating, and engagement score remain constant until changed
    last_compensation = row['Compensation']
    last_pay_raise_date = row['Compensation 2 date']
    variable_pay = row['Compensation 1']
    performance_rating = row['Review 2']
    engagement_score = row['Engagement 2']
    
    # Fill missing values with the last known values
    if pd.isna(row['Compensation']):
        row['Compensation'] = last_compensation
    if pd.isna(row['Compensation 2 date']):
        row['Compensation 2 date'] = last_pay_raise_date
    if pd.isna(row['Compensation 1']):
        row['Compensation 1'] = variable_pay
    if pd.isna(row['Review 2']):
        row['Review 2'] = performance_rating
    if pd.isna(row['Engagement 2']):
        row['Engagement 2'] = engagement_score
    
    output_df = output_df.append({
        'Employee Code': row['Employee Code'],
        'Manager Employee': row['Manager Employee Code'],
        'Last Compensation': last_compensation,
        'Compensation': row['Compensation'],
        'Last Pay Raise Date': last_pay_raise_date,
        'Variable Pay': row['Compensation 1'],
        'Tenure in Org': (row['Date of Exit'] - row['Date of Joining']).days if not pd.isna(row['Date of Exit']) else None,
        'Performance Rating': row['Review 2'],
        'Engagement Score': row['Engagement 2'],
        'Effective Date': row['Date of Joining'],
        'End Date': row['End Date']
    }, ignore_index=True)



In [8]:
output_df

,Employee Code,Manager Employee,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,1,NaN,20000,20000,NaN,NaN,None,NaN,NaN,2021-01-01,2100-01-01
1,2,1.0,20000,20000,2023-01-01,10000.0,None,9.5,5.0,2021-01-01,2100-01-01
2,3,1.0,20000,20000,2023-01-01,10000.0,1094,9.5,5.0,2021-01-01,2100-01-01


In [ ]:
output_df.to_csv(output_file, index=False, columns=['Employee Code', 'Manager', 'Last Compensation', 'Compensation',
                                                    'Last Pay Raise Date', 'Variable Pay', 'Tenure in Org',
                                                    'Performance Rating', 'Engagement Score', 'Effective Date', 'End Date'])